In [ ]:
# Step 1: OCR with PaddleOCR
from paddleocr import PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')
result = ocr.ocr('./Images/bill.jpg', cls=True)

ocr_text = "\n".join([line[1][0] for block in result for line in block])
print("📄 OCR Extracted Text:\n", ocr_text)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Load FLAN-T5 small model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

# Prompt
prompt = f"""Extract merchant, date, total amount, items with quantity and price, and payment mode from this receipt text:\n\n{ocr_text}\n\nReturn result as JSON."""

# Tokenize and generate
inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
outputs = model.generate(**inputs, max_new_tokens=200)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)


In [1]:
# Step 1: OCR with PaddleOCR
from paddleocr import PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')
result = ocr.ocr('./Images/cloth_bill.jpg', cls=True)

ocr_text = "\n".join([line[1][0] for block in result for line in block])
print("📄 OCR Extracted Text:\n", ocr_text)


[2025/04/07 15:34:10] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/Users/sriyansketavarapu/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/Users/sriyansketavarapu/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3,

In [5]:
import re
from pprint import pprint
def extract_bill_info(text):
    lines = text.splitlines()
    items = []
    merchant = None
    date = None
    time = None
    total_amount = None
    payment_mode = None
    change = None

    # 1. Date & Time
    date_pattern = re.compile(r'\d{2}-\d{2}-\d{4}')
    time_pattern = re.compile(r'\d{1,2}:\d{2}\s?(AM|PM)?', re.IGNORECASE)

    # 2. Items
    item_pattern = re.compile(r'(\d+)x\s?([A-Za-z\s]+)\n?\$([0-9]+\.[0-9]{2})')

    # 3. Total
    total_pattern = re.compile(r'\$([0-9]+\.[0-9]{2})')

    for i, line in enumerate(lines):
        if not line.strip():
            continue

        # Merchant
        if not merchant and "VectorStock" in line:
            merchant = "VectorStock"

        # Date
        if not date:
            match = date_pattern.search(line)
            if match:
                date = match.group()

        # Time
        if not time:
            match = time_pattern.search(line)
            if match:
                time = match.group()

        # Items
        if "x" in line and "$" in lines[i+1]:
            qty_item = line.strip()
            price = lines[i+1].strip()
            m = re.match(r'(\d+)x\s?(.+)', qty_item)
            if m:
                qty, item = m.groups()
                items.append({
                    "item": item.strip(),
                    "quantity": int(qty),
                    "price": float(price.replace('$', ''))
                })

        # Total
        if "TOTAL" in line.upper():
            match = total_pattern.search(lines[i+1])
            if match:
                total_amount = float(match.group(1))

        # Payment mode
        if "CASH" in line.upper():
            payment_mode = "Cash"

        if "CARD" in line.upper() or "Bank Card" in line:
            payment_mode = "Card"

        # Change
        if "CHANGE" in line.upper():
            match = total_pattern.search(lines[i+1])
            if match:
                change = float(match.group(1))

    return {
        "merchant": merchant,
        "date": date,
        "time": time,
        "items": items,
        "total_amount": total_amount,
        "payment_mode": payment_mode,
        "change_given": change
    }

pprint(extract_bill_info(ocr_text))


{'change_given': 26.12,
 'date': '09-10-2018',
 'items': [{'item': 'T-Shirt', 'price': 21.9, 'quantity': 1},
           {'item': 'T-Shirt', 'price': 12.99, 'quantity': 1},
           {'item': 'Pants', 'price': 35.99, 'quantity': 1},
           {'item': 'Socks', 'price': 4.0, 'quantity': 1}],
 'merchant': 'VectorStock',
 'payment_mode': 'Card',
 'time': '10:49AM',
 'total_amount': 74.88}
